In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import urllib.request
from sklearn import preprocessing
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
# 훈련 데이터 다운로드
#urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/multinli.train.ko.tsv", filename="multinli.train.ko.tsv")
#urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/snli_1.0_train.ko.tsv", filename="snli_1.0_train.ko.tsv")

In [3]:
# 검증 데이터 다운로드
#urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.dev.ko.tsv", filename="xnli.dev.ko.tsv")

In [4]:
#테스트 데이터 다운로드
#urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.test.ko.tsv", filename="xnli.test.ko.tsv")

In [5]:
train_snli = pd.read_csv("snli_1.0_train.ko.tsv", sep='\t', quoting=3)
train_xnli = pd.read_csv("multinli.train.ko.tsv", sep='\t', quoting=3)
val_data = pd.read_csv("xnli.dev.ko.tsv", sep='\t', quoting=3)
test_data = pd.read_csv("xnli.test.ko.tsv", sep='\t', quoting=3)

In [6]:
train_snli.head()

,sentence1,sentence2,gold_label
0,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 경쟁을 위해 말을 훈련시키고 있다.,neutral
1,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 식당에서 오믈렛을 주문하고 있다.,contradiction
2,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,사람은 야외에서 말을 타고 있다.,entailment
3,카메라에 웃고 손을 흔드는 아이들,그들은 부모님을 보고 웃고 있다,neutral
4,카메라에 웃고 손을 흔드는 아이들,아이들이 있다,entailment


In [7]:
train_xnli.head()

,sentence1,sentence2,gold_label
0,개념적으로 크림 스키밍은 제품과 지리라는 두 가지 기본 차원을 가지고 있다.,제품과 지리학은 크림 스키밍을 작동시키는 것이다.,neutral
1,시즌 중에 알고 있는 거 알아? 네 레벨에서 다음 레벨로 잃어버리는 거야 브레이브스...,사람들이 기억하면 다음 수준으로 물건을 잃는다.,entailment
2,우리 번호 중 하나가 당신의 지시를 세밀하게 수행할 것이다.,우리 팀의 일원이 당신의 명령을 엄청나게 정확하게 실행할 것이다.,entailment
3,어떻게 아세요? 이 모든 것이 다시 그들의 정보다.,이 정보는 그들의 것이다.,entailment
4,"그래, 만약 네가 테니스화 몇 개를 사러 간다면, 나는 왜 그들이 100달러대에서 ...",테니스화의 가격은 다양하다.,neutral


In [8]:
# 결합 후 섞기
train_data = train_snli.append(train_xnli)
train_data = train_data.sample(frac=1) #셔플링 100%

C:\Users\DanielJeong\AppData\Local\Temp\ipykernel_4860\989561213.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_snli.append(train_xnli)


In [9]:
train_data.head()

,sentence1,sentence2,gold_label
321592,"방은 매우 시크하고, 완전한 오디오 시스템을 갖추고 있다.",방에는 시대에 뒤떨어진 유행에 뒤떨어진 감각이 있다.,contradiction
168761,마드모아젤 신시아를 보호하고 있었어요.”,그는 신시아 마드모아젤을 보호하는데 실패했다.,neutral
328484,검은 정장을 입은 남자가 흰 정장과 검은 드레스를 입은 사람들 앞에 서 있다.,검은 옷을 입은 남자가 쇼를 하고 있다.,neutral
524919,지팡이를 든 남자가 걷고 있다.,남자는 소파에 앉아 차를 마셨다.,contradiction
351090,한 남자가 서서 동료들과 토론하고 있다,남자가 동료들 근처에 조용히 서 있다.,contradiction


In [10]:
val_data.head()

,sentence1,sentence2,gold_label
0,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 학교 버스가 그를 내려주자마자 엄마에게 전화를 걸었다.,neutral
1,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 한마디도 하지 않았다.,contradiction
2,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 엄마에게 집에 갔다고 말했다.,entailment
3,내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 ...,나는 워싱턴에 가본 적이 없어서 거기 배정을 받았을 때 그 장소를 찾으려다가 길을 ...,neutral
4,내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 ...,워싱턴으로 진군하면서 해야 할 일이 무엇인지 정확히 알고 있었다.,contradiction


In [11]:
test_data.head()

,sentence1,sentence2,gold_label
0,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",나는 그와 다시 이야기하지 않았다.,contradiction
1,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",나는 다시 그와 이야기를 하기 시작했다는 것에 너무 화가 났다.,entailment
2,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",우리는 좋은 대화를 나눴다.,neutral
3,"그리고 저는 그것이 특권이라고 생각했습니다, 그리고 여전히, 여전히, 당시 저는 A...",그날 현장에 나만 있었던 게 아니라는 걸 몰랐던 것이다.,neutral
4,"그리고 저는 그것이 특권이라고 생각했습니다, 그리고 여전히, 여전히, 당시 저는 A...",나는 AFFC 공군 경력 분야에서 그 번호를 가진 유일한 사람이라는 인상을 가지고 ...,entailment


In [12]:
def drop_na_and_duplicates(df):
    df = df.dropna()
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    return df

In [13]:
train_data = drop_na_and_duplicates(train_data)
val_data = drop_na_and_duplicates(val_data)
test_data = drop_na_and_duplicates(test_data)

In [14]:
# 서브워드 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

In [15]:
#최대 길이
max_seq_len=128

In [16]:
# 두개의 문장을 입력 받아야 한다.
def convert_example_to_feature(sent_list1, sent_list2, max_seq_len, tokenizer):
    input_ids, attention_masks, token_type_ids = [], [], []
    #1. wordpiece 임베딩 : 실질적인 입력
    #2. 실제 단어 어텐션 연산을 위한 어텐션 임베딩
    #3. 세그먼트 임베딩 = token_type_ids (문장 구분)
    #4. 포지셔널 임베딩 : 단어의 위치 정보를 표시  //128크기
    for sent1, sent2 in tqdm(zip(sent_list1, sent_list2), total = len(sent_list1)):
        encoding_result = tokenizer.encode_plus(sent1, sent2, max_length=max_seq_len, pad_to_max_length=True)
        #서브워드 토크나이저의 encode_plus 함수 동작
        # 1. 문장을 토크나이징
        # 2. 문장의 각 위치에 특수 토큰 추가
        # 3. 각 토큰을 인덱스(정수)로 변환
        # 4. max_length에 따라 문장의 길이를 맞추는 작업을 진행
        # 5. pad_to_max_length 기능을 통해 최대 문장 길이 까지 패딩
        # 6. 어텐션 마스크를 리턴
        # 7. 세그먼트 임베딩 벡터를 리턴
        
        input_ids.append(encoding_result['input_ids']) #
        attention_masks.append(encoding_result['attention_mask'])
        token_type_ids.append(encoding_result['token_type_ids'])

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    return (input_ids, attention_masks, token_type_ids)

In [17]:
X_train = convert_example_to_feature(train_data['sentence1'], train_data['sentence2'], max_seq_len = max_seq_len, tokenizer=tokenizer)
# sent1과 sent2를 인자로 3개의 임베딩 벡터 생성

  0%|                                                                               | 0/941814 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\DanielJeong\AppData\Roaming\Python\Python39\site-packages\transformers\tokenization_utils_base.py:2302: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model

  1%|▌                                                                 | 7978/941814 [00:02<04:44, 3287.15it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  1%|▌                                                                 | 8651/941814 [00:02<04:43, 3296.06it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs w

  2%|█▏                                                               | 18023/941814 [00:05<04:42, 3269.96it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  2%|█▌                                                               | 22407/941814 [00:06<04:36, 3329.17it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs w

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  3%|██▏                                                              | 32099/941814 [00:09<04:45, 3186.13it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have

  5%|███▌                                                             | 51726/941814 [00:15<04:25, 3357.51it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  6%|███▋                                                             | 53732/941814 [00:16<04:28, 3308.12it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs w

...

 94%|████████████████████████████████████████████████████████████    | 884675/941814 [04:31<00:16, 3393.27it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
 94%|████████████████████████████████████████████████████████████▏   | 885350/941814 [04:31<00:16, 3336.79it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs w

 96%|█████████████████████████████████████████████████████████████▎  | 901845/941814 [04:36<00:12, 3302.03it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
 96%|█████████████████████████████████████████████████████████████▎  | 902176/941814 [04:36<00:12, 3285.09it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs w

 97%|█████████████████████████████████████████████████████████████▉  | 911600/941814 [04:39<00:08, 3359.90it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
 97%|█████████████████████████████████████████████████████████████▉  | 911937/941814 [04:39<00:08, 3352.92it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs w

 98%|██████████████████████████████████████████████████████████████▋ | 922620/941814 [04:42<00:05, 3296.30it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
 99%|███████████████████████████████████████████████████████████████▏| 929607/941814 [04:44<00:03, 3211.67it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs w

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
100%|███████████████████████████████████████████████████████████████▊| 939178/941814 [04:47<00:00, 3392.74it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
100%|███████████████████████████████████████████████████████████████▊| 939855/941814 [04:47<00:00, 3327.01it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs w

In [18]:
#첫번째 문장의 인코딩 결과
input_id = X_train[0][0]
attention_mask = X_train[1][0]
token_type_id = X_train[2][0]

print('단어에 대한 정수 인코딩 : ', input_id)
print('어텐션 마스크 : ', attention_mask)
print('세그먼트 인코딩 : ', token_type_id)
print('각 인코딩 길이 : ', len(input_id))
print('정수 인코딩 복원 : ', tokenizer.decode(input_id))


단어에 대한 정수 인코딩 :  [    2  1129  2073  4230 11819 19521    16  5124  2470 14584  4119  2069
  5330  2088  1513  2062    18     3  1129  2170  2259  3891  2170 28788
  4847  6882  2170 28788  4847  5700  2052  1513  2062    18     3     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
어텐션 마스크 :  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [19]:
X_val = convert_example_to_feature(val_data['sentence1'], val_data['sentence2'], max_seq_len=max_seq_len, tokenizer=tokenizer )

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2490/2490 [00:00<00:00, 3003.63it/s]


In [50]:
# 최대 길이: 128
input_id = X_val[0][0]
attention_mask = X_val[1][0]
token_type_id = X_val[2][0]

print('단어에 대한 정수 인코딩 :',input_id)
print('어텐션 마스크 :',attention_mask)
print('세그먼트 인코딩 :',token_type_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))

단어에 대한 정수 인코딩 : [    2  3673   636  2116  1041  2371  2062    16     6  4122    16  1535
  1458 10283    18     6     3   636  2259  3741  4942  2116   636  2138
  4105  2223  2155  6000  4122  2170  2318  4117  2138   572  2359  2062
    18     3     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
어텐션 마스크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [20]:
X_test = convert_example_to_feature(test_data['sentence1'], test_data['sentence2'], max_seq_len=max_seq_len, tokenizer=tokenizer )

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5010/5010 [00:01<00:00, 2966.25it/s]


In [48]:
test_data.head()

,sentence1,sentence2,gold_label
0,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",나는 그와 다시 이야기하지 않았다.,contradiction
1,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",나는 다시 그와 이야기를 하기 시작했다는 것에 너무 화가 났다.,entailment
2,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",우리는 좋은 대화를 나눴다.,neutral
3,"그리고 저는 그것이 특권이라고 생각했습니다, 그리고 여전히, 여전히, 당시 저는 A...",그날 현장에 나만 있었던 게 아니라는 걸 몰랐던 것이다.,neutral
4,"그리고 저는 그것이 특권이라고 생각했습니다, 그리고 여전히, 여전히, 당시 저는 A...",나는 AFFC 공군 경력 분야에서 그 번호를 가진 유일한 사람이라는 인상을 가지고 ...,entailment


In [47]:
X_test[0][0]

array([   2, 8911,   16,  717, 2259, 3724, 2170, 9149, 3628, 2446, 2232,
       1889, 2118, 1380, 2886, 3683,   16,  717, 2259, 3760, 8642, 2371,
       2088,   16, 3983,  636, 2170, 2318, 3690, 3758, 2205, 2318,  859,
       2359, 2062,   18,    3,  717, 2259,  636, 2522, 3690, 3758, 2205,
       2118, 1380, 2886, 2062,   18,    3,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0])

In [49]:
X_test[0][1]

array([   2, 8911,   16,  717, 2259, 3724, 2170, 9149, 3628, 2446, 2232,
       1889, 2118, 1380, 2886, 3683,   16,  717, 2259, 3760, 8642, 2371,
       2088,   16, 3983,  636, 2170, 2318, 3690, 3758, 2205, 2318,  859,
       2359, 2062,   18,    3,  717, 2259, 3690,  636, 2522, 3758, 2138,
       1889, 2015, 3670, 2371, 4000,  575, 2170, 3760, 9562,  726, 2062,
         18,    3,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0])

In [21]:
np.unique(train_data.gold_label)

array(['contradiction', 'entailment', 'neutral'], dtype=object)

In [22]:
train_label = train_data['gold_label'].tolist()
val_label = val_data['gold_label'].tolist()
test_label = test_data['gold_label'].tolist()

In [23]:
#레이블을 정수 인덱스로 변환
idx_encode = preprocessing.LabelEncoder()
#레이블 객체 레이블 학습
idx_encode.fit(train_label)

y_train = idx_encode.transform(train_label)
y_val = idx_encode.transform(val_label)
y_test = idx_encode.transform(test_label)

In [24]:
idx_encode.classes_

array(['contradiction', 'entailment', 'neutral'], dtype='<U13')

In [25]:
label_idx = dict(zip(list(idx_encode.classes_), idx_encode.transform(list(idx_encode.classes_))))
print(label_idx)

idx_label = {value : key for key, value in label_idx.items()}
idx_label

{'contradiction': 0, 'entailment': 1, 'neutral': 2}


{0: 'contradiction', 1: 'entailment', 2: 'neutral'}

In [26]:
# 분류를 위한 트랜스포머 bert모델 로드
from transformers import TFBertForSequenceClassification
from tensorflow.keras.callbacks import EarlyStopping

In [27]:
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=3, from_pt=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
early_stopping = EarlyStopping(
    monitor="val_accuracy", 
    min_delta=0.001,
    patience=2)

model.fit(
    X_train, y_train, epochs=2, batch_size=16, validation_data = (X_val, y_val),
    callbacks = [early_stopping]
)

Epoch 1/2
58864/58864 [==============================] - 41445s 704ms/step - loss: 0.5631 - accuracy: 0.7738 - val_loss: 0.6101 - val_accuracy: 0.7514
Epoch 2/2
58864/58864 [==============================] - 41313s 702ms/step - loss: 0.4852 - accuracy: 0.8104 - val_loss: 0.6192 - val_accuracy: 0.7466


In [31]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test, batch_size=512)[1]))

10/10 [==============================] - 56s 6s/step - loss: 0.5921 - accuracy: 0.7553

 테스트 정확도: 0.7553


In [65]:
print(len(X_test[0])) #test_dataset의 워드피스임베딩벡터
print(len(X_test[1])) #test_dataset의 attention_masks
len(X_test[2]) #test_dataset의 token_type_ids 

5010
5010


5010

In [66]:
len(test_data)

5010

In [57]:
X_test[0][:2]

array([[   2, 8911,   16,  717, 2259, 3724, 2170, 9149, 3628, 2446, 2232,
        1889, 2118, 1380, 2886, 3683,   16,  717, 2259, 3760, 8642, 2371,
        2088,   16, 3983,  636, 2170, 2318, 3690, 3758, 2205, 2318,  859,
        2359, 2062,   18,    3,  717, 2259,  636, 2522, 3690, 3758, 2205,
        2118, 1380, 2886, 2062,   18,    3,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],
       [   2, 8911,   16,  717, 2259, 3724, 2170, 9149, 3628, 2446, 2232,
        1889, 2118, 1380, 2886, 3683,   16,  717, 2259, 3760,

  0%|                                                                                                                                                                                                          | 0/5 [00:00<?, ?it/s]C:\Users\DanielJeong\AppData\Roaming\Python\Python39\site-packages\transformers\tokenization_utils_base.py:2302: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,

In [73]:

sampleData1 = np.array(['나는 생각한다.', '나는 생각한다.', '나는 생각한다.'])
tmp1 = pd.Series(sampleData1)

sampleData2 = np.array(['고로 존재한다.', '고로 죽을을 맞이한다.', '그것은 플라스틱 물체이다.'])
tmp2 = pd.Series(sampleData2)

In [75]:
ex = convert_example_to_feature(tmp1, tmp2, max_seq_len, tokenizer)

  0%|                                                                                                                                                                                                          | 0/3 [00:00<?, ?it/s]C:\Users\DanielJeong\AppData\Roaming\Python\Python39\site-packages\transformers\tokenization_utils_base.py:2302: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,

In [89]:
result = model.predict(ex)
for i in range(len(ex[0])):
    print(idx_label[np.argmax(result[0][i])])

neutral
neutral
neutral
